In [11]:
import sys
sys.path.append('../src')
from config import *

In [12]:
import melt
melt.init_flags()
FLAGS = melt.get_flags()
FLAGS.pretrained = '../input/tf-xlm-roberta-base'

In [13]:
from model import xlm_model as Model

In [14]:
model = Model()

I0502 11:12:13.773058 140559559972672 logging.py:97] load xlm_model from ../input/tf-xlm-roberta-base start
I0502 11:12:13.774057 140559559972672 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0502 11:12:13.774940 140559559972672 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length"

In [15]:
melt.print_model(model)

Model: "tf_roberta_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) ((None, None, 768), (None 278043648 
Total params: 278,043,648
Trainable params: 278,043,648
Non-trainable params: 0
_________________________________________________________________
Model: "pooling_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_pooling_1 (FirstPoolin (None, 768)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Model: "xlm_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_roberta_model_2 (TFRobert ((None, None, 768), (None 278043648 
_________________________________________________________________


In [16]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, None)]            0         
_________________________________________________________________
xlm_model_1 (XlmModel)       (None, 1)                 278044417 
Total params: 278,044,417
Trainable params: 278,044,417
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.layers[1].summary()

Model: "xlm_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_roberta_model_2 (TFRobert ((None, None, 768), (None 278043648 
_________________________________________________________________
pooling_1 (Pooling)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 769       
Total params: 278,044,417
Trainable params: 278,044,417
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.layers[1].layers[0]

In [18]:
import gezi
from transformers import TFAutoModel
def xlm_model():
  pretrained = FLAGS.pretrained 
  with gezi.Timer(f'load xlm_model from {pretrained}', True, print):
    transformer = TFAutoModel.from_pretrained(pretrained)
  input_word_ids = tf.keras.Input(shape=(FLAGS.max_len,), dtype=tf.int32, name="input_word_ids")
  sequence_output = transformer(input_word_ids)[0]
  cls_token = sequence_output[:, 0, :]
  odim = len(toxic_types) + 1 if FLAGS.multi_head else 1
  out = tf.keras.layers.Dense(odim, activation='sigmoid')(cls_token)

  model = tf.keras.Model(inputs=input_word_ids, outputs=out)

  return model
model = xlm_model()

I0502 11:12:20.275863 140559559972672 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0502 11:12:20.276938 140559559972672 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "min_length": 0,
  "model_type": "xlm-roberta",
  "no_repeat_ngram

load xlm_model from ../input/tf-xlm-roberta-base start
load xlm_model from ../input/tf-xlm-roberta-base duration: 2.7378575801849365


In [ ]:
model.layers

In [ ]:
root='../working/exps/v2/toxic-multi-en-finetune-unint-large'
model.load_weights(f'{root}/model.h5')

In [ ]:
os.system(f'mkdir -p {root}/tf-xlm-roberta-large')

In [ ]:
model.layers[1].save_pretrained(f'{root}/tf-xlm-roberta-large')

In [ ]:
model.layers[3].save_weights('f{root}/dense.h5')

In [ ]:
model.layers[1].layers[0]

In [ ]:
os.system(f'mkdir -p {root}')
model.layers[-1].layers[0].save_pretrained(f'{root}/tf-xlm-roberta-base')

In [ ]:
model.layers[1].save_weights('../working/exps/v2/toxic-multi-en-finetune-unint/lang_model_weight.h5')

In [ ]:
FLAGS.pretrained = '../input/tf-xlm-roberta-large'
model = Model()
model.load_weights('../working/exps/v2/toxic-multi-en-finetune-unint-large/model.h5')

In [ ]:
model.layers[1].save_weights('../working/exps/v2/toxic-multi-en-finetune-unint-large/lang_model_weight.h5')

In [ ]:
model.weights[-1]

In [ ]:
model.load_weights('../working/exps/v1/debug/model_weight.h5')
model.weights[-1]

In [ ]:
import gezi
from dataset import Dataset
ds = Dataset('valid').make_batch(32, gezi.list_files('../input/tfrecords/xlm/validation'))
next(iter(ds))

In [ ]:
from evaluate import evaluate
from husky.callbacks import EvalCallback
eval_callback = EvalCallback(model, ds, evaluate)

In [ ]:
eval_callback.eval()

In [ ]:
import tensorflow as tf
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/debug'))
model.weights[-1]